**Author**: CodeForAll  
**License**: MIT License

--- 


## 🧠 Routh-Hurwitz Stability Criterion

The **Routh-Hurwitz criterion** is used to determine the **stability** of a linear time-invariant (LTI) system without solving for the roots of its characteristic equation.

---

### ✅ Characteristic Equation

Given a system with characteristic equation:

$$
a_0 s^n + a_1 s^{n-1} + a_2 s^{n-2} + \cdots + a_n = 0
$$

The **Routh-Hurwitz criterion** tells us the number of roots in the **right half of the complex plane (RHP)** by constructing a special table (Routh Array).

---

### 🧾 Stability Condition

- The system is **stable** if **all the roots** of the characteristic equation lie in the **left-half of the s-plane**.
- This happens **if and only if all the elements in the first column of the Routh array have the same sign** (no sign change).

---

### 🧮 Routh Array Construction

1. First two rows:
   - Row 1: Coefficients of even powers: $a_0, a_2, a_4, \dots$
   - Row 2: Coefficients of odd powers: $a_1, a_3, a_5, \dots$

2. Subsequent rows are computed using:

$$
R_{i,j} = \frac{
    R_{i-1,0} \cdot R_{i-2,j+1} - R_{i-2,0} \cdot R_{i-1,j+1}
}{
    R_{i-1,0}
}
$$

---

### ⚠️ Special Cases

1. **Zero in the first column**: Replace with a small $\epsilon > 0$.
2. **Entire row is zero**: Use the derivative of the auxiliary polynomial from the previous row.

---

### 📌 Example

Given:

$$
s^3 + 3s^2 + 3s + 1 = 0
$$

Routh array:

| s³ | 1 | 3 |
|----|---|---|
| s² | 3 | 1 |
| s¹ | 2 | 0 |
| s⁰ | 1 | 0 |

All first-column elements are **positive** ⇒ **stable system**.

----




In [ ]:
from sympy import symbols, Matrix, simplify, limit, Symbol, sympify, sign

def routh_hurwitz_symbolic(coeffs):
    eps = Symbol('ε')  # small symbolic value
    coeffs = [sympify(c) for c in coeffs]

    n = len(coeffs)
    rows = n
    cols = (n + 1) // 2
    routh = [[0 for _ in range(cols)] for _ in range(rows)]

    # First two rows
    routh[0] = coeffs[::2] + [0]*(cols - len(coeffs[::2]))
    routh[1] = coeffs[1::2] + [0]*(cols - len(coeffs[1::2]))

    # Fill out the Routh array
    for i in range(2, rows):
        for j in range(cols - 1):
            a = routh[i - 2][0]
            b = routh[i - 2][j + 1]
            c = routh[i - 1][0]
            d = routh[i - 1][j + 1]

            c = c if c != 0 else eps  # avoid divide by zero
            routh[i][j] = simplify(((c * b) - (a * d)) / c)

        # Row of zeros case
        if all(r == 0 for r in routh[i]):
            order = n - i
            prev_row = routh[i - 1]
            aux_poly = [prev_row[k] * (order - 2 * k) for k in range(len(prev_row))]
            routh[i] = aux_poly + [0]*(cols - len(aux_poly))

    # Make symbolic matrix
    routh_matrix = Matrix(routh)

    # Sign change detection
    first_col = [sympify(row[0]) for row in routh]
    first_col_eval = [limit(val.subs(eps, 1e-6), eps, 0, dir='+') for val in first_col]
    signs = [int(sign(x)) for x in first_col_eval if x != 0]

    sign_changes = sum(signs[i] != signs[i + 1] for i in range(len(signs) - 1))

    return routh_matrix, sign_changes


In [9]:
from sympy import init_printing

init_printing()
coeffs = [1, -3, 2]

routh_array, unstable_poles = routh_hurwitz_symbolic(coeffs)
print("Routh Array:")
print(routh_array)
print(f"Number of right-half-plane poles: {unstable_poles}")


Routh Array:
Matrix([[1, 2], [-3, 0], [2, 0]])
Number of right-half-plane poles: 2


In [10]:
from sympy import init_printing

init_printing()
coeffs = [1, 2, 3, 4, 5]

routh_array, unstable_poles = routh_hurwitz_symbolic(coeffs)
print("Routh Array:")
print(routh_array)
print(f"Number of right-half-plane poles: {unstable_poles}")

Routh Array:
Matrix([[1, 3, 5], [2, 4, 0], [1, 5, 0], [-6, 0, 0], [5, 0, 0]])
Number of right-half-plane poles: 2


In [11]:
from sympy import init_printing

init_printing()
coeffs = [1, 6, 15, 20, 15, 6, 1]

routh_array, unstable_poles = routh_hurwitz_symbolic(coeffs)
print("Routh Array:")
print(routh_array)
print(f"Number of right-half-plane poles: {unstable_poles}")

Routh Array:
Matrix([[1, 15, 15, 1], [6, 20, 6, 0], [35/3, 14, 1, 0], [64/5, 192/35, 0, 0], [9, 1, 0, 0], [256/63, 0, 0, 0], [1, 0, 0, 0]])
Number of right-half-plane poles: 0
